### Import Packages

In [39]:
import pandas as pd
import numpy as np
import datetime

### Import Raw Data from URL

In [3]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
data = pd.read_csv(url)

### Analysing specific data


In [6]:
data.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '7/13/20', '7/14/20', '7/15/20', '7/16/20', '7/17/20', '7/18/20',
       '7/19/20', '7/20/20', '7/21/20', '7/22/20'],
      dtype='object', length=187)

In [7]:
data.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,34455,34740,34994,35070,35229,35301,35475,35526,35615,35727
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3571,3667,3752,3851,3906,4008,4090,4171,4290,4358
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,19689,20216,20770,21355,21948,22549,23084,23691,24278,24872
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,858,861,862,877,880,880,880,884,884,889
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,525,541,576,607,638,687,705,749,779,812


#### Check the total number of rows and columns

In [8]:
data.shape

(266, 187)

>There are total 266 rows and 187 columns

### Rename the columns

In [20]:
covid_data = data.rename(
        columns = {'Province/State':'subregion'
                   ,'Country/Region':'country'
                   ,'Lat':'lat'
                   ,'Long':'long'
                   }
        )


In [21]:
print(covid_data)

    subregion                country        lat       long  1/22/20  1/23/20  \
0         NaN            Afghanistan  33.939110  67.709953        0        0   
1         NaN                Albania  41.153300  20.168300        0        0   
2         NaN                Algeria  28.033900   1.659600        0        0   
3         NaN                Andorra  42.506300   1.521800        0        0   
4         NaN                 Angola -11.202700  17.873900        0        0   
..        ...                    ...        ...        ...      ...      ...   
261       NaN  Sao Tome and Principe   0.186400   6.613100        0        0   
262       NaN                  Yemen  15.552727  48.516388        0        0   
263       NaN                Comoros -11.645500  43.333300        0        0   
264       NaN             Tajikistan  38.861000  71.276100        0        0   
265       NaN                Lesotho -29.610000  28.233600        0        0   

     1/24/20  1/25/20  1/26/20  1/27/20

### Reshaping the data 

The dates all exist as separate columns, which is not needed.

For further analysis of the data, it is required that all the dates exist under a single  “date” column.

>Here I have used Pandas melt function. Pandas has two functions for transposing data, melt and pivot, but melt is used when we need to transform data from wide form to long form.

In [22]:

covid_data = (covid_data.melt(id_vars = ['country','subregion','lat','long']
                 ,var_name = 'date_RAW'
                 ,value_name = 'confirmed'
                 )
)


>Two new variables are created: date_RAW and confirmed.
>The first is a date field, and the second is the number of confirmed cases.

In [23]:
print(covid_data)

                     country subregion        lat       long date_RAW  \
0                Afghanistan       NaN  33.939110  67.709953  1/22/20   
1                    Albania       NaN  41.153300  20.168300  1/22/20   
2                    Algeria       NaN  28.033900   1.659600  1/22/20   
3                    Andorra       NaN  42.506300   1.521800  1/22/20   
4                     Angola       NaN -11.202700  17.873900  1/22/20   
...                      ...       ...        ...        ...      ...   
48673  Sao Tome and Principe       NaN   0.186400   6.613100  7/22/20   
48674                  Yemen       NaN  15.552727  48.516388  7/22/20   
48675                Comoros       NaN -11.645500  43.333300  7/22/20   
48676             Tajikistan       NaN  38.861000  71.276100  7/22/20   
48677                Lesotho       NaN -29.610000  28.233600  7/22/20   

       confirmed  
0              0  
1              0  
2              0  
3              0  
4              0  
...      

>The confirmed variable is the number of confirmed covid-19 cases, for a particular place, on a particular date.

>The date_RAW variable is a string-based date variable. This needs certain more further modification in order to convert the dates into proper date/time data. 

In [24]:
(covid_data
    .filter(['date_RAW'])
)

,date_RAW
0,1/22/20
1,1/22/20
2,1/22/20
3,1/22/20
4,1/22/20
...,...
48673,7/22/20
48674,7/22/20
48675,7/22/20
48676,7/22/20


>The dates are in a form : one-digit month, two-digit day, and a two-digit year.
These need to be converted into proper datetime data.

In [25]:
# TEST DATE CONVERSION

(covid_data
    .assign(date = pd.to_datetime(covid_data.date_RAW, format='%m/%d/%y'))
    .filter(['date','date_RAW','confirmed'])
    .groupby(['date','date_RAW'])
    .agg('sum')
    .sort_values('date')
)

,,confirmed
date,date_RAW,
2020-01-22,1/22/20,555
2020-01-23,1/23/20,654
2020-01-24,1/24/20,941
2020-01-25,1/25/20,1434
2020-01-26,1/26/20,2118
...,...,...
2020-07-18,7/18/20,14292942
2020-07-19,7/19/20,14507589
2020-07-20,7/20/20,14714367


>The code created a new variable in the output called date.
Then I retrieved only a few columns (date, date_RAW, confirmed) using the filter method.
Then I grouped and aggregated the data.

Now that I tested this date conversion, I’ll run it properly and save the output so that I have my new date.

### Create a DATE Variable

> I have createed a copy of the date, just to back it up.

In [27]:
# BACKUP
#covid_data_backup_BEFOREDATE = covid_data.copy()


In [28]:
# CONVERT DATE
#-------------
covid_data = covid_data.assign(
         date = pd.to_datetime(covid_data.date_RAW, format='%m/%d/%y')
         )

Now, the covid_data contains a date variable that’s formatted as a proper datetime.

###  Rearange columns & sort

I have rearranged the columns with filter, and sorted the data with sort_values.

In [31]:
# SORT & REARANGE DATA
#=====================
covid_data = (covid_data
               .filter(['country', 'subregion', 'date', 'lat', 'long', 'confirmed'])
               .sort_values(['country','subregion','lat','long','date'])
               )

In [32]:
print(covid_data)

           country subregion       date        lat       long  confirmed
0      Afghanistan       NaN 2020-01-22  33.939110  67.709953          0
266    Afghanistan       NaN 2020-01-23  33.939110  67.709953          0
532    Afghanistan       NaN 2020-01-24  33.939110  67.709953          0
798    Afghanistan       NaN 2020-01-25  33.939110  67.709953          0
1064   Afghanistan       NaN 2020-01-26  33.939110  67.709953          0
...            ...       ...        ...        ...        ...        ...
47578     Zimbabwe       NaN 2020-07-18 -19.015438  29.154857       1478
47844     Zimbabwe       NaN 2020-07-19 -19.015438  29.154857       1611
48110     Zimbabwe       NaN 2020-07-20 -19.015438  29.154857       1713
48376     Zimbabwe       NaN 2020-07-21 -19.015438  29.154857       1820
48642     Zimbabwe       NaN 2020-07-22 -19.015438  29.154857       2034

[48678 rows x 6 columns]


### Setting the Index

I have set the index to ‘country‘. This was temporary to retrieve data based on country name.

To set the index for the DataFrame, I have used the Pandas set_index method.

In [33]:
# SET INDEX
covid_data.set_index('country', inplace = True)

I want to get a list of the country names, because currently country has been used as the index.

To do this, I have chained together several Pandas methods, including reset_index, filter, and drop_duplicates.

Essentially, I have creayed a list of the unique values of country.

In [34]:
# GET COUNTRY NAMES bcz country is used as the index
pd.set_option('display.max_rows', 155)

In [35]:
(covid_data
    .reset_index()
    .filter(['country'])
    .drop_duplicates()
    .head(n = 200)
)

,country
0,Afghanistan
183,Albania
366,Algeria
549,Andorra
732,Angola
...,...
47763,West Bank and Gaza
47946,Western Sahara
48129,Yemen
48312,Zambia


In [37]:
pd.reset_option('display.max_rows')

In [38]:
# PULL DATA FOR UNITED STATES
covid_data.loc['US']

,subregion,date,lat,long,confirmed
country,,,,,
US,NaN,2020-01-22,40.0,-100.0,1
US,NaN,2020-01-23,40.0,-100.0,1
US,NaN,2020-01-24,40.0,-100.0,2
US,NaN,2020-01-25,40.0,-100.0,2
US,NaN,2020-01-26,40.0,-100.0,5
...,...,...,...,...,...
US,NaN,2020-07-18,40.0,-100.0,3711413
US,NaN,2020-07-19,40.0,-100.0,3773260
US,NaN,2020-07-20,40.0,-100.0,3834677
